this set up was made with the help of this guide https://nbviewer.org/github/microsoft/AIforEarthDataSets/blob/main/data/sentinel-5p.ipynb#Auth-files

# Environment setup

In [1]:
import os
import fsspec
import json
import urllib3
import xarray as xr
import numpy as np
from azure.storage.blob import ContainerClient
from datetime import datetime, timezone

# Not used directly, but needs to be installed to read NetCDF files with xarray
import h5netcdf

# Constants

In [2]:
# Let's look at ozone concentration from mid-day on Jan 1, 2021
product = 'L2__O3____'
date = '2021/01/01'

# get Token from URL

In [3]:
def get_Token_Data():
    http = urllib3.PoolManager()
    response = http.request('GET', 'https://planetarycomputer.microsoft.com/api/sas/v1/token/sentinel5euwest/sentinel-5p')
    data = json.loads(response.data.decode('utf-8'))
    return data

# Check Token File

## helper functions

In [4]:
def is_file_empty(file_path):
    # check if file exist and is empty
    return os.path.exists(file_path) and os.stat(file_path).st_size == 0

def getTime():
    dt = datetime().now().astimezone(timezone.pst)
    dt_string = dt.isoformat(timespec = 'milliseconds').replace('+00:00', 'Z')
    return dt_string
    
def checkToken(jsonData):
    token_expiration = jsonData['msft:expiry']
    currTime = getTime()
    if token_expiration <= currTime :
        print('token expired! expiration Time: ', currTime)
        return 0
    else:
        print('token still Valid! expiration Time ', curTime)
        return 1

## Read Token

In [5]:
sas_path = './tokens/sentinel-5p_sas.json'
is_empty = is_file_empty(sas_path)

if is_empty != 0:
    with open(sas_path, 'r+') as f:
        data = json.load(f)
        if checkToken(data):
            sas_token=data['token']
        else:
            newData = get_Token_Data()
            sas_token = newData['token']
            f.seek(0)
            json.dump(newData, f)
else:
    with open(sas_path, 'r+') as f:
        newData = get_Token_Data()
        sas_token = newData['token']
        f.seek(0)
        json.dump(newData, f)

# Azure storage constants

In [6]:
storage_account_name = 'sentinel5euwest'
container_name = 'sentinel-5p'
storage_account_url = 'https://' + storage_account_name + '.blob.core.windows.net/'

container_client = ContainerClient(account_url=storage_account_url, 
                                                container_name=container_name,
                                                credential=sas_token)

# List products matching our product/date

In [7]:
prefix = '/'.join(['TROPOMI',product,date])
print('Searching for prefix {}'.format(prefix))
generator = container_client.list_blobs(name_starts_with=prefix)
scene_paths = [blob.name for blob in generator]
print('\nFound {} matching scenes:\n'.format(len(scene_paths)))
for s in scene_paths:
    print(s.split('/')[-1])

Searching for prefix TROPOMI/L2__O3____/2021/01/01

Found 14 matching scenes:

S5P_OFFL_L2__O3_____20210101T013036_20210101T031206_16680_01_020104_20210102T182728.nc
S5P_OFFL_L2__O3_____20210101T031206_20210101T045336_16681_01_020104_20210102T203532.nc
S5P_OFFL_L2__O3_____20210101T045336_20210101T063506_16682_01_020104_20210102T224247.nc
S5P_OFFL_L2__O3_____20210101T063506_20210101T081636_16683_01_020104_20210103T000719.nc
S5P_OFFL_L2__O3_____20210101T081636_20210101T095806_16684_01_020104_20210103T013023.nc
S5P_OFFL_L2__O3_____20210101T095806_20210101T113936_16685_01_020104_20210103T031528.nc
S5P_OFFL_L2__O3_____20210101T113936_20210101T132106_16686_01_020104_20210103T052522.nc
S5P_OFFL_L2__O3_____20210101T132106_20210101T150236_16687_01_020104_20210103T065107.nc
S5P_OFFL_L2__O3_____20210101T150236_20210101T164406_16688_01_020104_20210103T081248.nc
S5P_OFFL_L2__O3_____20210101T164406_20210101T182536_16689_01_020104_20210103T101745.nc
S5P_OFFL_L2__O3_____20210101T182536_20210101T200706

# Print Metadata for one scene

In [8]:
offl_scenes = [s for s in scene_paths if 'OFFL' in s]
scene_path = offl_scenes[len(offl_scenes) // 2]
url = storage_account_url + container_name + '/' + scene_path
print('Processing image at URL:\n{}'.format(url))

Processing image at URL:
https://sentinel5euwest.blob.core.windows.net/sentinel-5p/TROPOMI/L2__O3____/2021/01/01/S5P_OFFL_L2__O3_____20210101T132106_20210101T150236_16687_01_020104_20210103T065107/S5P_OFFL_L2__O3_____20210101T132106_20210101T150236_16687_01_020104_20210103T065107.nc


# IT BREAKS HERE NEED TO FIND A WAY TO AUTHORIZE THE ACCESS TO THE URL WITH OUR TOKEN

In [11]:
import warnings; warnings.filterwarnings('ignore')
with fsspec.open(url+sas_token) as f:
    ds = xr.open_dataset(f)
print(ds)

FileNotFoundError: https://sentinel5euwest.blob.core.windows.net/sentinel-5p/TROPOMI/L2__O3____/2021/01/01/S5P_OFFL_L2__O3_____20210101T132106_20210101T150236_16687_01_020104_20210103T065107/S5P_OFFL_L2__O3_____20210101T132106_20210101T150236_16687_01_020104_20210103T065107.ncst=2022-03-19T21%3A24%3A23Z&se=2022-03-20T22%3A09%3A23Z&sp=rl&sv=2020-06-12&sr=c&skoid=c85c15d6-d1ae-42d4-af60-e2ca0f81359b&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2022-03-20T20%3A24%3A23Z&ske=2022-03-21T21%3A24%3A23Z&sks=b&skv=2020-06-12&sig=qPpm%2B0/ThdPPUghpnUKqtX4U2g0FSJAIPt0G/Zd6IZM%3D

# Open the Data

In [12]:
with fsspec.open(url+sas_token) as f:
    ds = xr.open_dataset(f,group='/PRODUCT')
print(ds)

FileNotFoundError: https://sentinel5euwest.blob.core.windows.net/sentinel-5p/TROPOMI/L2__O3____/2021/01/01/S5P_OFFL_L2__O3_____20210101T132106_20210101T150236_16687_01_020104_20210103T065107/S5P_OFFL_L2__O3_____20210101T132106_20210101T150236_16687_01_020104_20210103T065107.ncst=2022-03-19T21%3A24%3A23Z&se=2022-03-20T22%3A09%3A23Z&sp=rl&sv=2020-06-12&sr=c&skoid=c85c15d6-d1ae-42d4-af60-e2ca0f81359b&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2022-03-20T20%3A24%3A23Z&ske=2022-03-21T21%3A24%3A23Z&sks=b&skv=2020-06-12&sig=qPpm%2B0/ThdPPUghpnUKqtX4U2g0FSJAIPt0G/Zd6IZM%3D